# Intro to Gragh Convolutional Surrogate Models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('/home/ewhalen/projects/caeSurrogateUtility')
import caeUtility as cu

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=100, 
                     batch_size=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/debugging/')

plotHistory(history)

epoch: 0   trainLoss: 9.2937e-01   valLoss:9.4691e-01  time: 1.31e+00
epoch: 1   trainLoss: 8.4876e-01   valLoss:9.4595e-01  time: 1.08e+00
epoch: 2   trainLoss: 8.3690e-01   valLoss:9.4772e-01  time: 1.04e+00
epoch: 3   trainLoss: 8.0642e-01   valLoss:9.4307e-01  time: 1.04e+00
epoch: 4   trainLoss: 7.6164e-01   valLoss:9.4097e-01  time: 1.03e+00
epoch: 5   trainLoss: 6.9443e-01   valLoss:9.3580e-01  time: 1.03e+00
epoch: 6   trainLoss: 6.8161e-01   valLoss:9.0224e-01  time: 1.05e+00
epoch: 7   trainLoss: 6.5083e-01   valLoss:8.2340e-01  time: 1.11e+00
epoch: 8   trainLoss: 6.0626e-01   valLoss:7.4623e-01  time: 1.03e+00
epoch: 9   trainLoss: 5.9062e-01   valLoss:6.7410e-01  time: 1.04e+00
epoch: 10   trainLoss: 5.6345e-01   valLoss:6.2706e-01  time: 1.12e+00
epoch: 11   trainLoss: 5.3510e-01   valLoss:6.0369e-01  time: 1.19e+00
epoch: 12   trainLoss: 5.2143e-01   valLoss:5.9595e-01  time: 1.05e+00
epoch: 13   trainLoss: 4.7612e-01   valLoss:5.7132e-01  time: 1.00e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000015,0.002612,0.119007,0.894488,0.952675,0.399118,-5.952305
test,0.000023,0.002958,0.130521,0.717306,0.887011,0.382444,-4.842859


## 5. Visualize some predictions

In [15]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)